In [6]:
import csv
from datetime import datetime
from pysnmp.hlapi import *

# SNMP nastavení
target_ip = '192.168.1.1'
community = 'public'
port = 161
oids = {
    'ISP_IN': '1.3.6.1.2.1.2.2.1.10.2',     # příklad OID pro Rx
    'LAN_OUT': '1.3.6.1.2.1.2.2.1.16.3',    # příklad OID pro Tx
    'ISP_SPEED': '1.3.6.1.2.1.2.2.1.5.2'    # příklad OID pro speed (bajty/s)
}

def snmp_get(oid):
    iterator = getCmd(
        SnmpEngine(),
        CommunityData(community, mpModel=0),
        UdpTransportTarget((target_ip, port), timeout=3, retries=2),
        ContextData(),
        ObjectType(ObjectIdentity(oid))
    )
    errorIndication, errorStatus, errorIndex, varBinds = next(iterator)
    if errorIndication or errorStatus:
        return None
    for varBind in varBinds:
        return int(varBind[1])

def bytes_to_mb(value):
    return round(value / (1024 * 1024), 2)

def bytes_to_mbps(value):
    return round((value * 8) / (1024 * 1024), 2)

# Získání dat
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
isp_in_mb = bytes_to_mb(snmp_get(oids['ISP_IN']))
lan_out_mb = bytes_to_mb(snmp_get(oids['LAN_OUT']))
isp_speed_mbps = bytes_to_mbps(snmp_get(oids['ISP_SPEED']))

# Zápis do CSV
with open('traffic_log.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([timestamp, isp_in_mb, lan_out_mb, isp_speed_mbps])



TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [ ]:
import sys
print(sys.executable)

/Users/Marek/miniconda3/envs/snmp_env_310/bin/python


In [ ]:
from pysnmp.hlapi import *
print("✅ pysnmp funguje!")


✅ pysnmp funguje!


In [ ]:
!pip list | grep pysnmp


pysnmp            7.1.21


In [ ]:
from pysnmp.hlapi import getCmd
print("✅ getCmd funguje")




✅ getCmd funguje


In [ ]:
import pysnmp
print(pysnmp.__file__)


/Users/Marek/miniconda3/envs/snmp_env_310/lib/python3.10/site-packages/pysnmp/__init__.py


In [ ]:
from pysnmp.hlapi import *

target_ip = '192.168.11.100'
community = 'omegadejdar'
port = 161

def snmp_walk(oid):
    results = []
    for (errorIndication, errorStatus, errorIndex, varBinds) in nextCmd(
        SnmpEngine(),
        CommunityData(community, mpModel=0),
        UdpTransportTarget((target_ip, port), timeout=3, retries=2),
        ContextData(),
        ObjectType(ObjectIdentity(oid)),
        lexicographicMode=False
    ):
        if errorIndication or errorStatus:
            print("SNMP chyba:", errorIndication or errorStatus)
            break
        for varBind in varBinds:
            results.append((str(varBind[0]), varBind[1].prettyPrint()))
    return results

# Získání názvů rozhraní
if_descr_list = snmp_walk('1.3.6.1.2.1.2.2.1.2')

# Výpis
for oid, name in if_descr_list:
    print(f"{oid} → {name}")


1.3.6.1.2.1.2.2.1.2.1 → lo
1.3.6.1.2.1.2.2.1.2.2 → eth0
1.3.6.1.2.1.2.2.1.2.3 → gre0
1.3.6.1.2.1.2.2.1.2.4 → wifi1
1.3.6.1.2.1.2.2.1.2.5 → teql0
1.3.6.1.2.1.2.2.1.2.6 → wifi0
1.3.6.1.2.1.2.2.1.2.7 → ath0
1.3.6.1.2.1.2.2.1.2.8 → br0
1.3.6.1.2.1.2.2.1.2.9 → airview1


In [ ]:
from pysnmp.hlapi import *

target_ip = '192.168.11.100'
community = 'omegadejdar'
oid = '1.3.6.1.2.1.2.2.1.2'  # ifDescr

for (errorIndication, errorStatus, errorIndex, varBinds) in nextCmd(
    SnmpEngine(),
    CommunityData(community),
    UdpTransportTarget((target_ip, 161)),
    ContextData(),
    ObjectType(ObjectIdentity(oid)),
    lexicographicMode=False
):
    if errorIndication:
        print(f"Chyba: {errorIndication}")
        break
    elif errorStatus:
        print(f"{errorStatus.prettyPrint()} at {errorIndex}")
        break
    else:
        for varBind in varBinds:
            print(f"{varBind[0]} = {varBind[1]}")



Chyba: No SNMP response received before timeout
